In [1]:
#importing the libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM , Dense,GlobalMaxPooling1D,Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

In [2]:
#mounting the drive

In [3]:
#importing the dataset
with open('/content/drive/MyDrive/cv1/content.json') as content:
  data1 = json.load(content)
#getting all the data to lists
tags = []
inputs = []
responses={}
for intent in data1['intents']:
  responses[intent['tag']]=intent['responses']
  for lines in intent['input']:
    inputs.append(lines)
    tags.append(intent['tag'])
#converting to dataframe
data = pd.DataFrame({"inputs":inputs,
                     "tags":tags})
print(data)

                         inputs       tags
0                  hello friend   greeting
1                     hi friend   greeting
2                         hello   greeting
3                            hi   greeting
4                     hey there   greeting
5                           hey   greeting
6                      hi there   greeting
7          hi, nice to meet you   greeting
8          hi, nice to meet you   greeting
9                        hi bot   greeting
10                    hello bot   greeting
11                   hi chatbot   greeting
12                      hey bot   greeting
13                      hi dude   greeting
14                   hello dude   greeting
15                          bye    goodbye
16                     okay bye    goodbye
17                      goodbye    goodbye
18                see you later    goodbye
19       i will catch you later    goodbye
20        i will catch up later    goodbye
21    okay i will see you later    goodbye
22     I wi

In [4]:
#removing punctuations
import string
data['inputs'] = data['inputs'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['inputs'] = data['inputs'].apply(lambda wrd: ''.join(wrd))
#tokenize the data
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['inputs'])
train = tokenizer.texts_to_sequences(data['inputs'])

#apply padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)

#encoding the outputs
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])

In [5]:
#input length
input_shape = x_train.shape[1]
print(input_shape)
#define vocabulary
vocabulary = len(tokenizer.word_index)
print("number of unique words : ",vocabulary)
#output length
output_length = le.classes_.shape[0]
print("output length: ",output_length)

6
number of unique words :  42
output length:  4


In [6]:
#creating the model
i = Input(shape=(input_shape,))
x = Embedding(vocabulary+1,10)(i)
x = LSTM(10,return_sequences=True)(x)
x = Flatten()(x)
x = Dense(output_length,activation="softmax")(x)
model  = Model(i,x)
#compiling the model
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
#training the model
train = model.fit(x_train,y_train,epochs=200)

Epoch 1/200
2/2 [==============================] - 0s 4ms/step - loss: 1.3874 - accuracy: 0.2683
Epoch 2/200
2/2 [==============================] - 0s 3ms/step - loss: 1.3847 - accuracy: 0.4390
Epoch 3/200
2/2 [==============================] - 0s 3ms/step - loss: 1.3822 - accuracy: 0.4146
Epoch 4/200
2/2 [==============================] - 0s 3ms/step - loss: 1.3800 - accuracy: 0.3659
Epoch 5/200
2/2 [==============================] - 0s 3ms/step - loss: 1.3778 - accuracy: 0.3659
Epoch 6/200
2/2 [==============================] - 0s 4ms/step - loss: 1.3759 - accuracy: 0.3659
Epoch 7/200
2/2 [==============================] - 0s 3ms/step - loss: 1.3736 - accuracy: 0.3659
Epoch 8/200
2/2 [==============================] - 0s 3ms/step - loss: 1.3715 - accuracy: 0.3659
Epoch 9/200
2/2 [==============================] - 0s 3ms/step - loss: 1.3691 - accuracy: 0.3659
Epoch 10/200
2/2 [==============================] - 0s 3ms/step - loss: 1.3669 - accuracy: 0.3659
Epoch 11/200
2/2 [===========

In [7]:
#chatting
import random
while True:
  texts_p = []
  prediction_input = input('You : ')
  #removing punctuation and converting to lowercase
  prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
  prediction_input = ''.join(prediction_input)
  texts_p.append(prediction_input)
  #tokenizing and padding
  prediction_input = tokenizer.texts_to_sequences(texts_p)
  prediction_input = np.array(prediction_input).reshape(-1)
  prediction_input = pad_sequences([prediction_input],input_shape)
  #getting output from model
  output = model.predict(prediction_input)
  output = output.argmax()
  #finding the right tag and predicting
  response_tag = le.inverse_transform([output])[0]
  print("Cv1: ",random.choice(responses[response_tag]))
  if response_tag == "goodbye":
    break

You : Heyy
Cv1:  Hello my friend, How can I help you ?
You : What are you?
Cv1:  You can call me cv1 and I am here to Help you
You : Oh okayy. Then good night!!
Cv1:  Hi
You : Then good night
Cv1:  Hello my friend, How can I help you ?
You : Good bye
Cv1:  Yeah Sure, Will talk to you later
